In [63]:
# imports
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import der Basisdaten

### HIER MÜSSEN WIR NOCH EINMAL FESTHALTEN WOHER UNSERE DATEN KOMMEN UND DAS SAUBER AUFLISTEN

In [187]:
# data imports
df_data = pd.read_csv("data/Modellierung_SoSe23.csv", sep=";", decimal=",", usecols=[0,1,4,5])
pv_data = pd.read_csv("data/ninja_pv_50.8263_6.9721.csv", decimal=".", header=3)


# fill dataframe
df_data['pv'] = pv_data['electricity'].to_numpy()

df_data

,Zeit,electric_load,heat_load,temp,pv
0,01.01. 00:00,0.016247,0.22,10.70,0.0
1,01.01. 01:00,0.016162,0.13,10.77,0.0
2,01.01. 02:00,0.016005,0.07,10.55,0.0
3,01.01. 03:00,0.166450,0.03,10.44,0.0
4,01.01. 04:00,0.016200,0.07,10.34,0.0
...,...,...,...,...,...
8755,31.12. 19:00,1.107500,1.69,0.83,0.0
8756,31.12. 20:00,0.261240,1.73,0.51,0.0
8757,31.12. 21:00,0.289830,1.33,0.20,0.0
8758,31.12. 22:00,0.340810,0.81,-0.12,0.0


In [188]:
# calculation and data preparation for heat_pump(wp)

# scraped data
wp_data = {
    "temp" : [-22, -15, -7, 2, 7, 10, 12, 20, 25, 30, 35],
    "heating_power" : [0, 25.50, 35.20, 43.00, 56.40, 60.30, 62.90, 67.30, 70.40, 73.60, 76.70],
    "el_power" : [0, 14.58, 15.54, 16.70, 18.54, 18.87, 18.87, 19.05, 19.13, 19.36, 19.46],
    "cop" : [0, 1.7, 2.3, 2.6, 3.0, 3.2, 3.3, 3.5, 3.7, 3.8, 3.9]
}

# assumptions
wp_el_p_nom = 12.5  # kW

# calculate COP
df_data["wp_COP_interp"] = np.round(np.interp(df_data['temp'], wp_data["temp"], wp_data["cop"]),3)
df_data["wp_power_consumption"] = np.round(np.interp(df_data['temp'], wp_data["temp"], wp_data["el_power"]),2)
df_data['wp_power_cons_norm'] = df_data["wp_power_consumption"] / wp_el_p_nom

df_data

,Zeit,electric_load,heat_load,temp,pv,wp_COP_interp,wp_power_consumption,wp_power_cons_norm
0,01.01. 00:00,0.016247,0.22,10.70,0.0,3.235,18.87,1.5096
1,01.01. 01:00,0.016162,0.13,10.77,0.0,3.238,18.87,1.5096
2,01.01. 02:00,0.016005,0.07,10.55,0.0,3.228,18.87,1.5096
3,01.01. 03:00,0.166450,0.03,10.44,0.0,3.222,18.87,1.5096
4,01.01. 04:00,0.016200,0.07,10.34,0.0,3.217,18.87,1.5096
...,...,...,...,...,...,...,...,...
8755,31.12. 19:00,1.107500,1.69,0.83,0.0,2.561,16.55,1.3240
8756,31.12. 20:00,0.261240,1.73,0.51,0.0,2.550,16.51,1.3208
8757,31.12. 21:00,0.289830,1.33,0.20,0.0,2.540,16.47,1.3176
8758,31.12. 22:00,0.340810,0.81,-0.12,0.0,2.529,16.43,1.3144


# Parameter- und Variablendefinition

### Komponenparameter
Für eine leichtere Lesbarkeit und einfachere Zuordnung aller festgelegten Basisparameter, werden Basisdaten in Dictionarys mit eindeutigen Keys definiert

### Basisvariablen 
Generelle Basisvariablen wie z.B. der Strompreis oder die Gebäudedefinition werden in einfachen globalen Variablen gespeichert

In [189]:
# basicvariables
# cost related variables
electricity_rate = 0.4 # €/kWh
infeed_rate = -0.08    # €/kWh
gas_price = 0.158

# building related variables
living_space = 157 #m²

# grid parameters


# data calculations 
# -> here <-------------------!


# component parameters
# PV-plant
pv_params = {
    'p_nom': 10,    # kWp
    'p_max_pu': df_data['pv'],
    'capital_cost': (1300*10)/20,
    'marginal_cost': 0.02 * 20,
    'lifespan' : 20    
}

# FuelCell -> fc
fc_params = {
    'p_nom': 1.5, #kW_el Brennstoffzellenleistung picea
    'el_efficiency': 0.8,
    'th_efficiency': 0.2,
    'capital_cost': 1000,
    'marginal_cost': 40
}

# Electrolyser -> ely
ely_params = {
    'p_nom': 2.3, #kW_el Wasserstofferzeugungsleistung picea
    'el_efficiency': 0.625,
    'th_efficiency': 0.2,
    'capital_cost': 12000,
    'marginal_cost': 0
}

# heatpump -> wp
wp_params = {
    'p_nom': 3,
    'efficiency': df_data['wp_COP_interp'],
    'capital_cost': (25000 / 7.5) / 25,
    'marginal_cost': 0
}

# battery storage
battery_params = { # battery integrated in picea system
    'e_nom': 20, #kWh picea system
    #'e_min_pu': 0.2,
    #'e_max_pu': 1.0,
    'capital_cost': 892/12.5,
    'marginal_cost': 0,
    'standing_loss': 0.02,
    'charge_efficiency': 0.98,
    'discharge_efficiency': 0.98,
}

# heat storage
heat_storage_params = {
    'e_nom': 300, #kWh
    'e_min_pu': 0.6,
    'e_max_pu': 1.0,
    'volume': 300, #Litre
    'capital_cost': 53.32/20,
    'marginal_cost': 0,
    'standing_loss': 0.08,
    'charge_efficiency': 0.95,
    'discharge_efficiency': 0.95,
}

# H2 gas storage
h2_storage_params = {
    'e_nom': 300, #kWh_el - picea, auf 1500 kWh_el erweiterbar
    #'e_min_pu_hydrogen': 0.6,
    #'e_max_pu_hydrogen': 1.0,
    'capital_cost': 120000, #€ picea system
    'marginal_cost': 499.8,
    'standing_loss': 0.001
}

# Klassendefinition für Szenarioanalyse

Mit dieser Klasse kann man eine neue Studie als Instanz initiieren. Die Instanz speichert dann alle Ergebnisdaten kompakt im Klassen-Objekt

In [190]:
class SzenarioAnalysis:
    def __init__(self):
        self.data = df_data
        self.results = []
        self.result_counter = len(self.results)
        n_ref = 0
        n_h2 = 0
        
    def build_ref_network(self):
        
        # initialyze network
        n_ref = pypsa.Network()
        n_ref.set_snapshots(range(8760))
        
        # add components
        
        # Bus es 
        # electrical busses
        n_ref.add('Bus', name='electricity')
        n_ref.add('Bus', name='bat_bus')    
        n_ref.add('Bus', name='house_IO')   # Überschussknoten -> Haus In- und Output
        # heat busses
        n_ref.add('Bus', name='heat')
        n_ref.add('Bus', name='heat_storage_bus')
        
        
        # Loads
        # electrical
        n_ref.add('Load', name='electric_load', bus='electricity',
                  p_set = self.data['electric_load'])
        # heat
        n_ref.add('Load', name='heat_load', bus='heat', 
                  p_set = self.data['heat_load'])
        
        # Generators
        # PV
        n_ref.add('Generator', name='pv', bus='house_IO',
                  p_nom = pv_params['p_nom'],
                  p_max_pu = pv_params['p_max_pu'],
                  capital_cost = pv_params['capital_cost'],
                  marginal_cost = pv_params['marginal_cost'])
        # grid  
        n_ref.add('Generator', name='infeed', bus='house_IO',
                  marginal_cost = infeed_rate,
                  sign = -1, 
                  p_nom_extendable = True)
        n_ref.add('Generator', name='grid', bus='house_IO',
                  marginal_cost = electricity_rate,
                  p_nom = self.data['electric_load'].max())
        
        
        # Storages
        n_ref.add('Store', name='battery',  bus='bat_bus',
                  capital_cost = battery_params['capital_cost'], 
                  standing_loss = battery_params['standing_loss'],
                  e_nom_extendable = True)
        n_ref.add('Store', name= 'heat_storage', bus='heat_storage_bus',
                  e_nom_max = 36.85,
                  capital_cost = heat_storage_params['capital_cost'],
                  standing_loss = heat_storage_params['standing_loss'],
                  e_nom_extendable = True)
        

        # Links
        n_ref.add('Link', name='charge_bat', bus0='electricity', bus1='bat_bus',
                  efficiency = battery_params['charge_efficiency'],
                  p_nom_extendable = True)
        n_ref.add('Link', name='discharge_bat', bus0='bat_bus', bus1='electricity',
                  efficiency = battery_params['discharge_efficiency'],
                  p_nom_extendable = True)
        n_ref.add('Link', name='housegrid', bus0='house_IO', bus1='electricity', 
                  p_nom_extendable = True)
        
        n_ref.add('Link', name='wp', bus0='electricity', bus1='heat',
                  efficiency = wp_params['efficiency'],
                  capital_cost= wp_params['capital_cost'],
                  p_nom_extendable = True)
        n_ref.add('Link', name='charge_heat', bus0='heat', bus1='heat_storage_bus',
                  efficiency = heat_storage_params['charge_efficiency'],
                  p_nom_extendable = True)
        n_ref.add('Link', name='discharge_heat', bus0='heat_storage_bus', bus1='heat',
                  efficiency = heat_storage_params['discharge_efficiency'],
                  p_nom_extendable = True)
        
        self.n_ref = n_ref
        print("reference network build")
    
    def calculate_results(self, network, sensitivity_variable=0):
        #Funktion der laufenden kosten

        #Funktion der laufende Kosten
        laufende_kosten_generators = (network.generators_t.p.grid * network.generators.marginal_cost.grid  +
                                           network.generators_t.p.infeed * network.generators.marginal_cost.infeed).sum()
        #Funktion für Investitionskosten
        #kapital_kosten_generators = (network.generators.p_nom_opt * network.generators.capital_cost).sum()      #nimmt hier auch Netzeinspeisung und Netzbezug mit rein
        kapital_kosten_generators = (network.generators.p_nom_opt.pv * network.generators.capital_cost.pv).sum()
        kapital_kosten_links = (network.links.p_nom_opt * network.links.capital_cost).sum()
        kapital_kosten_stores = (network.stores.e_nom_opt *network.stores.capital_cost).sum()
        kapital_kosten_gesamt = (kapital_kosten_stores + kapital_kosten_generators + kapital_kosten_links )

        #Gesamtkosten pro Jahr
        total_kosten = (laufende_kosten_generators + kapital_kosten_generators 
        + kapital_kosten_stores + kapital_kosten_links) 


        #Einspeisevergütung (separat betrachtet von Gesamtkosten)
        einspeise_ertrag = (network.generators_t.p.infeed).sum()*electricity_rate *-1


        #Jährliche Einsparung der marginalen Kosten
        bezug_ursprung = (network.loads_t.p.electric_load * network.generators.marginal_cost.grid +
                          network.loads_t.p.heat_load * gas_price).sum()
        einsparung = bezug_ursprung - laufende_kosten_generators 

        #Armotisation
        Anschaffungskosten = (kapital_kosten_generators *20) + (kapital_kosten_links *20) +(kapital_kosten_stores * 20) #+ (kapital_kosten_links *20)   # kapital_kosten_links doppelt in Rechnung
        Amortisation = Anschaffungskosten/einsparung 


        #relevante Ausgabewerte
        netzbezug_summe = (network.generators_t.p.grid).sum()
        netzeinspeisung_summe= (network.generators_t.p.infeed).sum()
        pv_ertrag_summe = (network.generators_t.p.pv).sum()
        einspeicherung_summe = (network.links_t.p0.charge_bat).sum()
        ausspeicherung_summe = (network.links_t.p0.discharge_bat).sum()
        Kapazität_Batterie = (network.stores.e_nom_opt.battery)
        Kapazität_Pufferspeicher =(network.stores.e_nom_opt.heat_storage)#*36.85)
        Leistung_wp = (network.links.p_nom_opt.wp)



        #Werte in Ergebnistabelle auflisten
        df_result = pd.concat([pd.DataFrame(data=total_kosten, index=[sensitivity_variable],columns=['Kapital- und marginale kosten €/a']),
                               pd.DataFrame(data=einsparung, index=[sensitivity_variable], columns=['Verbrauchseinsparung €/a']),
                               pd.DataFrame(data=laufende_kosten_generators, index=[sensitivity_variable], columns=['Marginale Kosten €/a']),
                               pd.DataFrame(data=bezug_ursprung, index=[sensitivity_variable], columns=['Ursprüngliche Energiekosten €/a']),
                               pd.DataFrame(data=kapital_kosten_gesamt, index=[sensitivity_variable], columns=['Investitionskosten €/a']),
                               pd.DataFrame(data=einspeise_ertrag, index=[sensitivity_variable], columns=['Einspeiseverguetung €/a']),
                               pd.DataFrame(data=Amortisation, index=[sensitivity_variable], columns=['Amortisationsdauer a']),
                               pd.DataFrame(data=netzbezug_summe, index=[sensitivity_variable], columns=['Netzbezug kWh/a']),
                               pd.DataFrame(data=netzeinspeisung_summe, index=[sensitivity_variable], columns=['Netzeinspeisung kWh/a']),
                               pd.DataFrame(data=pv_ertrag_summe, index=[sensitivity_variable], columns=['PV-Ertragssumme kWh/a']),
                               pd.DataFrame(data=einspeicherung_summe, index=[sensitivity_variable], columns=['Einspeicherung kWh/a']),
                               pd.DataFrame(data=ausspeicherung_summe, index=[sensitivity_variable], columns=['Auspeicherung kWh/a']),
                               pd.DataFrame(data=Kapazität_Pufferspeicher, index=[sensitivity_variable], columns=['Pufferspeicherkapazität kWh']),
                               pd.DataFrame(data=Kapazität_Batterie, index=[sensitivity_variable], columns=['Batteriepeicherkapazität kWh']),
                               pd.DataFrame(data=Leistung_wp, index=[sensitivity_variable], columns=['Leistung Wärmepumpe kW']),],
                               axis = 1)

        
        self.results.append(df_result)
        return df_result

In [191]:
def build_ref_network():

    # initialyze network
    n_ref = pypsa.Network()
    n_ref.set_snapshots(range(8760))

    # add components

    # Bus es 
    # electrical busses
    n_ref.add('Bus', name='electricity')
    n_ref.add('Bus', name='bat_bus')    
    n_ref.add('Bus', name='house_IO')   # Überschussknoten -> Haus In- und Output
    # heat busses
    n_ref.add('Bus', name='heat')
    n_ref.add('Bus', name='heat_storage_bus')


    # Loads
    # electrical
    n_ref.add('Load', name='electric_load', bus='electricity',
              p_set = df_data['electric_load'])
    # heat
    n_ref.add('Load', name='heat_load', bus='heat', 
              p_set = df_data['heat_load'])

    # Generators
    # PV
    n_ref.add('Generator', name='pv', bus='house_IO',
              p_nom = pv_params['p_nom'],
              p_max_pu = pv_params['p_max_pu'],
              capital_cost = pv_params['capital_cost'],
              marginal_cost = pv_params['marginal_cost'])
    # grid  
    n_ref.add('Generator', name='infeed', bus='house_IO',
              marginal_cost = infeed_rate,
              sign = -1, 
              p_nom_extendable = True)
    n_ref.add('Generator', name='grid', bus='house_IO',
              marginal_cost = electricity_rate,
              p_nom = df_data['electric_load'].max())


    # Storages
    n_ref.add('Store', name='battery',  bus='bat_bus',
              capital_cost = battery_params['capital_cost'], 
              standing_loss = battery_params['standing_loss'],
              e_nom_extendable = True)
    n_ref.add('Store', name= 'heat_storage', bus='heat_storage_bus',
              e_nom_max = 36.85,
              capital_cost = heat_storage_params['capital_cost'],
              standing_loss = heat_storage_params['standing_loss'],
              e_nom_extendable = True)


    # Links
    n_ref.add('Link', name='charge_bat', bus0='electricity', bus1='bat_bus',
              efficiency = battery_params['charge_efficiency'],
              p_nom_extendable = True)
    n_ref.add('Link', name='discharge_bat', bus0='bat_bus', bus1='electricity',
              efficiency = battery_params['discharge_efficiency'],
              p_nom_extendable = True)
    n_ref.add('Link', name='housegrid', bus0='house_IO', bus1='electricity', 
              p_nom_extendable = True)

    n_ref.add('Link', name='wp', bus0='electricity', bus1='heat',
              efficiency = wp_params['efficiency'],
              capital_cost= wp_params['capital_cost'],
              p_nom_extendable = True)
    n_ref.add('Link', name='charge_heat', bus0='heat', bus1='heat_storage_bus',
              efficiency = heat_storage_params['charge_efficiency'],
              p_nom_extendable = True)
    n_ref.add('Link', name='discharge_heat', bus0='heat_storage_bus', bus1='heat',
              efficiency = heat_storage_params['discharge_efficiency'],
              p_nom_extendable = True)
    
    return n_ref
    

In [200]:
def auswertungsfunktion(network, sensitivity_variable=0):
    #Funktion der laufenden kosten

    #Funktion der laufende Kosten
    laufende_kosten_generators = (network.generators_t.p.grid * network.generators.marginal_cost.grid  +
                                       network.generators_t.p.infeed * network.generators.marginal_cost.infeed).sum()
    #Funktion für Investitionskosten
    #kapital_kosten_generators = (network.generators.p_nom_opt * network.generators.capital_cost).sum()      #nimmt hier auch Netzeinspeisung und Netzbezug mit rein
    kapital_kosten_generators = (network.generators.p_nom_opt.pv * network.generators.capital_cost.pv).sum()
    kapital_kosten_links = (network.links.p_nom_opt * network.links.capital_cost).sum()
    kapital_kosten_stores = (network.stores.e_nom_opt *network.stores.capital_cost).sum()
    kapital_kosten_gesamt = (kapital_kosten_stores + kapital_kosten_generators + kapital_kosten_links )

    #Gesamtkosten pro Jahr
    total_kosten = (laufende_kosten_generators + kapital_kosten_generators 
    + kapital_kosten_stores + kapital_kosten_links) 


    #Einspeisevergütung (separat betrachtet von Gesamtkosten)
    einspeise_ertrag = (network.generators_t.p.infeed).sum()*electricity_rate *-1


    #Jährliche Einsparung der marginalen Kosten
    bezug_ursprung = (network.loads_t.p.electric_load * network.generators.marginal_cost.grid +
                      network.loads_t.p.heat_load * gas_price).sum()
    einsparung = bezug_ursprung - laufende_kosten_generators 

    #Armotisation
    Anschaffungskosten = (kapital_kosten_generators *20) + (kapital_kosten_links *20) +(kapital_kosten_stores * 20) #+ (kapital_kosten_links *20)   # kapital_kosten_links doppelt in Rechnung
    Amortisation = Anschaffungskosten/einsparung 


    #relevante Ausgabewerte
    netzbezug_summe = (network.generators_t.p.grid).sum()
    netzeinspeisung_summe= (network.generators_t.p.infeed).sum()
    pv_ertrag_summe = (network.generators_t.p.pv).sum()
    einspeicherung_summe = (network.links_t.p0.charge_bat).sum()
    ausspeicherung_summe = (network.links_t.p0.discharge_bat).sum()
    Kapazität_Batterie = (network.stores.e_nom_opt.battery)
    Kapazität_Pufferspeicher =(network.stores.e_nom_opt.heat_storage)#*36.85)
    Leistung_wp = (network.links.p_nom_opt.wp)



    #Werte in Ergebnistabelle auflisten
    df_result = pd.concat([pd.DataFrame(data=total_kosten, index=[sensitivity_variable],columns=['Kapital- und marginale kosten €/a']),
                           pd.DataFrame(data=einsparung, index=[sensitivity_variable], columns=['Verbrauchseinsparung €/a']),
                           pd.DataFrame(data=laufende_kosten_generators, index=[sensitivity_variable], columns=['Marginale Kosten €/a']),
                           pd.DataFrame(data=bezug_ursprung, index=[sensitivity_variable], columns=['Ursprüngliche Energiekosten €/a']),
                           pd.DataFrame(data=kapital_kosten_gesamt, index=[sensitivity_variable], columns=['Investitionskosten €/a']),
                           pd.DataFrame(data=einspeise_ertrag, index=[sensitivity_variable], columns=['Einspeiseverguetung €/a']),
                           pd.DataFrame(data=Amortisation, index=[sensitivity_variable], columns=['Amortisationsdauer a']),
                           pd.DataFrame(data=netzbezug_summe, index=[sensitivity_variable], columns=['Netzbezug kWh/a']),
                           pd.DataFrame(data=netzeinspeisung_summe, index=[sensitivity_variable], columns=['Netzeinspeisung kWh/a']),
                           pd.DataFrame(data=pv_ertrag_summe, index=[sensitivity_variable], columns=['PV-Ertragssumme kWh/a']),
                           pd.DataFrame(data=einspeicherung_summe, index=[sensitivity_variable], columns=['Einspeicherung kWh/a']),
                           pd.DataFrame(data=ausspeicherung_summe, index=[sensitivity_variable], columns=['Auspeicherung kWh/a']),
                           pd.DataFrame(data=Kapazität_Pufferspeicher, index=[sensitivity_variable], columns=['Pufferspeicherkapazität kWh']),
                           pd.DataFrame(data=Kapazität_Batterie, index=[sensitivity_variable], columns=['Batteriepeicherkapazität kWh']),
                           pd.DataFrame(data=Leistung_wp, index=[sensitivity_variable], columns=['Leistung Wärmepumpe kW']),],
                           axis = 1)

    return df_result

In [209]:
def calculate_results(network):
    
    # list of all timeseries resulting from optimization
    ts_results = []
    
    ts_buses = pd.DataFrame(network.buses_t)
    ts_gen = pd.DataFrame(network.generators_t)
    
    ts_results.append(ts_buses, ts_gen)
    
    
    power_supply = network.generators_t.p.grid
    netzeinspeisung_summe= network.generators_t.p.infeed
    pv_ertrag_summe = network.generators_t.p.pv
    charge_bat = network.links_t.p0.charge_bat
    ausspeicherung_summe = network.links_t.p0.discharge_bat
    battery_capacity = network.stores.e_nom_opt.battery
    Kapazität_Pufferspeicher =(network.stores.e_nom_opt.heat_storage)#*36.85)
    Leistung_wp = (network.links.p_nom_opt.wp)
    
    return ts_results

In [210]:
n_ref = build_ref_network()
n_ref.optimize(solver_name='gurobi')
res = calculate_results(n_ref)
res1 = auswertungsfunktion(n_ref, sensitivity_variable=0)

INFO:linopy.model: Solve linear problem using Gurobi solver
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 30.72it/s]


Read LP format model from file C:\Users\Marvin\AppData\Local\Temp\linopy-problem-hjbt3o_n.lp
Reading time = 0.37 seconds
obj: 254050 rows, 113889 columns, 473048 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 254050 rows, 113889 columns and 473048 nonzeros
Model fingerprint: 0xba15511b
Coefficient statistics:
  Matrix range     [9e-01, 4e+00]
  Objective range  [8e-02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 4e+01]
Presolve removed 188392 rows and 39470 columns
Presolve time: 0.26s
Presolved: 65658 rows, 74419 columns, 192630 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Dense cols : 3
 AA' NZ     : 1.270e+05
 Factor NZ  : 8.553e+05 (roughly 60 MB of memory

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 113889 primals, 254050 duals
Objective: 3.02e+03
Solver model: available
Solver message: 2



ValueError: If using all scalar values, you must pass an index

In [205]:
res.sum()

grid_supply [kW]               6696.292047
grid_infeed [kW]                  0.000000
pv_generated [kW]                 4.349111
battery_charge [kW]               0.000000
battery_discharge [kW]            0.000000
heat_storage_charge [kW]         17.265784
heat_storage_discharge [kW]      12.146919
dtype: float64

In [214]:
n_ref.generators_t

{'p_min_pu': Empty DataFrame
 Columns: []
 Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
 
 [8760 rows x 0 columns],
 'p_max_pu': Generator   pv
 snapshot      
 0          0.0
 1          0.0
 2          0.0
 3          0.0
 4          0.0
 ...        ...
 8755       0.0
 8756       0.0
 8757       0.0
 8758       0.0
 8759       0.0
 
 [8760 rows x 1 columns],
 'p_set': Empty DataFrame
 Columns: []
 Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 

In [212]:
df = pd.DataFrame(n_ref.buses_t.p)
df

Bus,electricity,bat_bus,house_IO,heat,heat_storage_bus
snapshot,,,,,
0,0.000000e+00,0.0,0.0,0.000000e+00,0.0
1,6.938894e-18,0.0,0.0,0.000000e+00,0.0
2,0.000000e+00,0.0,0.0,0.000000e+00,0.0
3,0.000000e+00,0.0,0.0,3.469447e-18,0.0
4,0.000000e+00,0.0,0.0,0.000000e+00,0.0
...,...,...,...,...,...
8755,0.000000e+00,0.0,0.0,0.000000e+00,0.0
8756,0.000000e+00,0.0,0.0,0.000000e+00,0.0
8757,-1.110223e-16,0.0,0.0,2.220446e-16,0.0
